In [1]:
import os
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()

root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
!ls data/

MNIST  mnist_m


In [2]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model

Using cache found in /home/ubuntu/.cache/torch/hub/ultralytics_yolov5_master
fatal: not a git repository (or any of the parent directories): .git
YOLOv5 🚀 2022-3-28 torch 1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
           

In [35]:
#list(model.named_parameters())
print(model.model.model.model[4])

C3(
  (cv1): Conv(
    (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (cv3): Conv(
    (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (m): Sequential(
    (0): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
    )
    (1): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
    )
  )
)


### Domain Adaptive YOLOv5s Network

The goal of this section is to introduce Gradient Reversal Layers (GRLs) into the YOLOv5s network.  The proposed architecture is based on MS_DAYOLO from [MULTISCALE DOMAIN ADAPTIVE YOLO FOR CROSS-DOMAIN OBJECT DETECTION](https://arxiv.org/pdf/2106.01483v2.pdf):

![MS-DAYOLO](MS-DAYOLO.png)

The following diagram shows more descriptive gradient flow on a YOLOv5s architecture diagram:

![YOLOv5s_w_DA](Yolov5_Arch_w_DA.png)

GRLs need to be added after each of the following:

model.model.model.model[4] <span style="color: red;">**-> F1** </span><br>
model.model.model.model[6] <span style="color: red;">**-> F2**</span>
<br>
model.model.model.model[8] <span style="color: red;">**-> F3**</span>

In [3]:
class MS_DAYOLOv5(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(DAYOLO, self).__init__()
        self.backbone = backbone
        
    def forward(self, input_images: torch.Tensor):
        return backbone(input_images, image_size=640)

In [ ]:
class ToBeHooked(nn.Module):
    def __init__(self):
        super(ToBeHooked, self).__init__()
        self.a = nn.Sequential(
            nn.Linear(1,3),
            nn.ReLU()
        )
        self.b = nn.Sequential(
            nn.Linear(3,5),
            nn.ReLU()
        )
        self.c = nn.Linear(5,1)
        
    def forward(self, x):
        out = self.a(x)
        out = self.b(out)
        out = self.c(out)
        
        return out
    
    
hook_me = ToBeHooked()
hook_me
for name, m in model.named_modules():
    print(name)


In [4]:
#for param in hook_me.b[0].parameters():
    #print(param)
    #mul = param*torch.ones(1,5)
    #print(f'Result: {mul}')
#parallel_out = hook_me.b(torch.ones(1,3))

# for GRL
from torch.autograd import Function

class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        # Store context for backprop
        ctx.alpha = alpha
        
        # Forward pass is a no-op
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        # Backward pass is just to -alpha the gradient
        output = grad_output.neg() * ctx.alpha
        
        # Must return same number as inputs to forward()
        return output, None

    
class NewModel(nn.Module):
    def __init__(self, backbone: nn.Module, output_layers):
        super().__init__()
        self.backbone = backbone
        self.output_layers = output_layers
        self.f1_domain_classifier = nn.Sequential(
            nn.Conv2d(64,)
        )
        self.selected_out = OrderedDict()
        self.fhooks = []
        
        #print(list(self.backbone.model.model.model._modules.keys()))
        
        for i,l in enumerate(list(self.backbone.model.model.model._modules.keys())):
            if i in self.output_layers:
                self.fhooks.append(getattr(self.backbone.model.model.model,l).register_forward_hook(self.forward_hook(l)))

    
    
    def forward_hook(self, layer_name):
        def hook(module, input, output):
            self.selected_out[layer_name] = output
        
        return hook
    
    def forward(self, x, grl_lambda):
        
        org_out = self.backbone(x)
        
        f1 = self.selected_out['4']
        grl_f1 = GradientReversalFn.apply(f1, grl_lambda)
        
        
        f2 = self.selected_out['6']
        frl_f2 = GradientReversalFn.apply(f2, grl_lambda)
        
        
        f3 = self.selected_out['8']
        grl_f3 = GradientReversalFn.apply(f3, grl_lambda)
        
        
        
        return org_out
    
    def available_layers(self):
        print(len(self.fhooks))
        for key in self.fhooks:
            print(f'Key: {key}')
        

new_model = NewModel(model, output_layers=[4,6,8]).to(device)   
new_model.available_layers()

TypeError: __init__() missing 2 required positional arguments: 'out_channels' and 'kernel_size'

In [32]:
print(next(model.parameters()).device)
batch = torch.rand(4,3,640,480, device=device)
print(batch.device)
#new_model(batch)

cuda:0
cuda:0


tensor([[[4.24362e+00, 4.57453e+00, 8.15290e+00,  ..., 9.65173e-04, 8.05810e-04, 3.39554e-03],
         [1.46842e+01, 5.75970e+00, 2.99006e+01,  ..., 1.05029e-03, 1.01106e-03, 3.56276e-03],
         [1.81727e+01, 6.26491e+00, 3.34709e+01,  ..., 1.28024e-03, 9.63305e-04, 2.63005e-03],
         ...,
         [4.02660e+02, 5.99369e+02, 1.47434e+02,  ..., 8.15852e-03, 1.15201e-03, 1.47859e-03],
         [4.28607e+02, 6.03118e+02, 1.07851e+02,  ..., 1.08494e-02, 1.43204e-03, 1.72749e-03],
         [4.53029e+02, 6.14867e+02, 1.13424e+02,  ..., 9.98767e-03, 1.52228e-03, 1.87308e-03]],

        [[4.44466e+00, 4.20125e+00, 8.98019e+00,  ..., 1.28940e-03, 7.14943e-04, 3.46511e-03],
         [1.20348e+01, 4.62148e+00, 2.42056e+01,  ..., 1.89548e-03, 8.95789e-04, 3.31240e-03],
         [1.79566e+01, 4.39180e+00, 3.05288e+01,  ..., 1.58493e-03, 9.73017e-04, 4.37087e-03],
         ...,
         [4.03375e+02, 5.99171e+02, 1.48627e+02,  ..., 9.31490e-03, 1.43981e-03, 2.04229e-03],
         [4.27087e+0

In [38]:
import torch.onnx

torch.onnx.export(model,
                 batch,
                 "YOLOv5s.onnx",
                 export_params=False,
                 input_names=['input'],
                 output_names=['output'],
                 opset_version=11)

/home/ubuntu/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:60: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.onnx_dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [39]:
!ls

MS-DAYOLO.png	      data					plot_utils.py
YOLOv5s.onnx	      dataAPI.py				yolov5s.pt
Yolov5_Arch_w_DA.png  domain_adaptation_mnist.ipynb
__pycache__	      domain_adaptation_object_detection.ipynb
